HomeWork 3. Мутации.
1. Возьмем небольшой кусок последовательности гена, с которым работали на предыдущих уроках. Внесем в него по очереди все типы мутаций (silence, missense, nonsense) и покажем их эффект.

In [1]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import MutableSeq
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Seq import MutableSeq
import time

In [4]:
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
gene = SeqIO.read('/content/drive/MyDrive/BIO/gene.fna', 'fasta')
gene.seq

Seq('ATACAAGAGATGTGAGAAGCACCATAAAAGGCGTTGTGAGGAGTTGTGGGGGAG...GTC')

In [8]:
# переходим с минуса на плюс-нить
gene = gene.seq.complement()
gene

Seq('TATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTC...CAG')

In [9]:
# находим старт-кодон "ATG"
start_codon_index = str(gene).find("ATG")
start_codon_index

1

In [10]:
gene[start_codon_index:]

Seq('ATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTCA...CAG')

In [11]:
# цепочка нуклеотидов со старт кодона
dna_seq = gene[start_codon_index:]
dna_seq

Seq('ATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTCA...CAG')

In [12]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

MutableSeq('ATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTCA...CAG')

**Nonsence** **mutation**

In [13]:
# заменим какой-нибудь кодон после стартового на стоп-кодон
stop_codon = "UAA"

In [14]:
stop = Seq('UAA')
stop.transcribe().translate()

Seq('*')

In [15]:
# рассмотрим белок из цепи ДНК dna_seq
protein = dna_seq.transcribe().translate()
protein

/usr/local/lib/python3.10/dist-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('MFSTLFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

In [16]:
mutable_seq[9:12] = stop_codon

In [17]:
nonsense_seq = mutable_seq.transcribe().translate()
nonsense_seq

MutableSeq('MFS*LFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

На 4-м триплете появился стоп-кодон.Белок синтезироваться не будет.

**Missense** **mutation**

In [18]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

MutableSeq('ATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTCA...CAG')

In [19]:
protein

Seq('MFSTLFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

In [20]:
# заменим аминокислоту на 4-й позиции
mutable_seq[9:12] = 'CAA'
missense_seq = mutable_seq.transcribe().translate()
missense_seq

MutableSeq('MFSQLFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

**Silent** **mutation**

In [21]:
mutable_seq = MutableSeq(dna_seq)
mutable_seq

MutableSeq('ATGTTCTCTACACTCTTCGTGGTATTTTCCGCAACACTCCTCAACACCCCCTCA...CAG')

In [22]:
protein

Seq('MFSTLFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

In [23]:
mutable_seq[9:12]

MutableSeq('ACA')

In [24]:
mutable_seq[9:12] = "ACC"

In [25]:
silent_seq = mutable_seq.transcribe().translate()
silent_seq

MutableSeq('MFSTLFVVFSATLLNTPSLPLFSNFSNNRRHAIFTRECPYPYIKIGRSLR*IGR...TSY')

Благодаря избыточенности генетического кода, замена одного нуктеотида не изменила синтез нужного белка.

2. Выясните сколько вариантов SNP с клинической значимостью есть для вашего гена.

In [26]:
import sys

In [37]:
Entrez.email = "dbsnp-user@nih.gov"
Entrez.api_key = None

eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          term='A2M[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class])',
                          # AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )
eSresult = Entrez.read(eShandle)
for k in eSresult:
    print (k, ":", eSresult[k])

Count : 0
RetMax : 0
RetStart : 0
QueryKey : 1
WebEnv : MCID_65b3a1ba1f5c995a06566d38
IdList : []
TranslationSet : []
TranslationStack : [{'Term': 'A2M[All Fields]', 'Field': 'All Fields', 'Count': '17597', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '130269', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '10256260', 'Explode': 'N'}, 'AND']
QueryTranslation : A2M[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class]


Для гена A2M патологические изменение не найдены в SNP.
Возьмем ген CAD.

In [38]:
Entrez.email = "dbsnp-user@nih.gov"
Entrez.api_key = None

eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          term='CAD[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class])',
                          # AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )

In [39]:
eSresult = Entrez.read(eShandle)

In [40]:
for k in eSresult:
    print (k, ":", eSresult[k])

Count : 4
RetMax : 4
RetStart : 0
QueryKey : 1
WebEnv : MCID_65b3a2ac3a8ecb78e70f40ba
IdList : ['1475014505', '766975615', '763410987', '751610198']
TranslationSet : []
TranslationStack : [{'Term': 'CAD[All Fields]', 'Field': 'All Fields', 'Count': '12552', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '130269', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '10256260', 'Explode': 'N'}, 'AND']
QueryTranslation : CAD[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class]


4 мутации для гена CAD есть в SNP. Патологические и несинанимичные.


In [45]:
from urllib.error import HTTPError

In [41]:
rslist = (eSresult['IdList'])

In [42]:
for rs in rslist:
    print(rs)

1475014505
766975615
763410987
751610198


In [43]:
webenv = eSresult["WebEnv"]
query_key = eSresult["QueryKey"]
total_count = int(eSresult["Count"])
query_key = eSresult["QueryKey"]
retmax = 2

In [46]:
fetch_count = 0
for start in range(0, total_count, retmax):
    end = min(total_count, start+retmax)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    #fetch_count += 1
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="snp",
                                         #rettype="uilist", #available types [uilist | xml (use retmode=xml))
                                         retmode="xml",
                                         retstart=start,
                                         retmax=retmax,
                                         webenv=webenv,
                                         query_key=query_key )
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    if (fetch_handle):
        #print(fetch_handle)
        data = fetch_handle.read()
        print(data)
        fetch_handle.close()


Going to download record 1 to 2
b'<?xml version="1.0" ?>\n<ExchangeSet xmlns:xsi="https://www.w3.org/2001/XMLSchema-instance" xmlns="https://www.ncbi.nlm.nih.gov/SNP/docsum" xsi:schemaLocation="https://www.ncbi.nlm.nih.gov/SNP/docsum ftp://ftp.ncbi.nlm.nih.gov/snp/specs/docsum_eutils.xsd" ><DocumentSummary uid="1475014505"><SNP_ID>1475014505</SNP_ID><ALLELE_ORIGIN/><GLOBAL_MAFS><MAF><STUDY>GnomAD</STUDY><FREQ>A=0.000007/1</FREQ></MAF><MAF><STUDY>TOMMO</STUDY><FREQ>T=0.00006/1</FREQ></MAF><MAF><STUDY>TOPMED</STUDY><FREQ>A=0.000004/1</FREQ></MAF><MAF><STUDY>ALFA</STUDY><FREQ>A=0./0</FREQ></MAF></GLOBAL_MAFS><GLOBAL_POPULATION/><GLOBAL_SAMPLESIZE>0</GLOBAL_SAMPLESIZE><SUSPECTED/><CLINICAL_SIGNIFICANCE>pathogenic</CLINICAL_SIGNIFICANCE><GENES><GENE_E><NAME>CAD</NAME><GENE_ID>790</GENE_ID></GENE_E></GENES><ACC>NC_000002.12</ACC><CHR>2</CHR><HANDLE>TOMMO_GENOMICS,GNOMAD,TOPMED</HANDLE><SPDI>NC_000002.12:27233314:G:A,NC_000002.12:27233314:G:T</SPDI><FXN_CLASS>coding_sequence_variant,missense_